In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import os
import glob
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pdb
from keras.utils import np_utils
from os import listdir
from os.path import isfile, join
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from keras.applications import VGG19
from keras.layers import *
from keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow import keras
from keras.layers import Dropout
from sklearn.utils import class_weight

print(tf.__version__)

2.4.1


In [2]:
# Data parameter
input_dir = os.path.join('..', 'input')
output_dir = os.path.join('..', 'output')

dataset_dir = os.path.join(input_dir, 'landmark-recognition-2020')
train_dir = os.path.join(dataset_dir, 'train')
train_labelmap_dir = os.path.join(dataset_dir, 'train.csv')
test_dir = os.path.join(dataset_dir, 'test')
test_labelmap_dir = os.path.join(dataset_dir, 'sample_submission.csv')

train_df = pd.read_csv(train_labelmap_dir)
test_df = pd.read_csv(test_labelmap_dir)
num_data = len(train_df)

In [3]:
train_dir

'../input/landmark-recognition-2020/train'

In [4]:
print(os.listdir(train_dir))

['7', '2', 'b', 'f', '5', 'e', '8', '0', 'a', '3', '1', 'c', '4', '9', '6', 'd']


In [5]:
train_df.head(10)

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7
5,28b13f94a6f1f3c1,7
6,307d6584f473ba35,7
7,4a7ba9eb16d51bc4,7
8,597353dfbb3df649,7
9,a40d00dc4fcc3a10,7


In [31]:
#Adding two extra columns to the data-frame, where filename will be used to retreive the images when needed, and the "label" is the integer version of the "landmark_id"
train_df["filename"] = train_df.id.str[0]+"/"+train_df.id.str[1]+"/"+train_df.id.str[2]+"/"+train_df.id+".jpg"
train_df["label"] = train_df.landmark_id.astype(str)

#Due to the limited resources, i will be keeping only the top 1000 classes of the data.
from collections import Counter

number_of_classes_to_keep = 200
c = train_df.landmark_id.values
count = Counter(c).most_common(number_of_classes_to_keep)
keep_labels = [i[0] for i in count]
train_keep = train_df[train_df.landmark_id.isin(keep_labels)]

#Since we took the 1000 most common classes, we will reset the index of the samples in addition to shuffling them.
train_keep = train_keep.sample(frac=1).reset_index(drop=True)
print(train_keep)

                     id  landmark_id                    filename   label
0      cb860df1330c0315        15445  c/b/8/cb860df1330c0315.jpg   15445
1      9c7d7c07b9771b72       138982  9/c/7/9c7d7c07b9771b72.jpg  138982
2      001f7aea9cb982ed        42123  0/0/1/001f7aea9cb982ed.jpg   42123
3      8c1cc81ce76a264e        31361  8/c/1/8c1cc81ce76a264e.jpg   31361
4      88110cb035cfb4af       149980  8/8/1/88110cb035cfb4af.jpg  149980
...                 ...          ...                         ...     ...
98435  9de1da9e35c0fadc       113838  9/d/e/9de1da9e35c0fadc.jpg  113838
98436  c8c4d0f4418cab53         1924  c/8/c/c8c4d0f4418cab53.jpg    1924
98437  2c3e4b07d9647918       173511  2/c/3/2c3e4b07d9647918.jpg  173511
98438  4212bd8696c63bdc       125786  4/2/1/4212bd8696c63bdc.jpg  125786
98439  3f040e9b8f7e29c1       113462  3/f/0/3f040e9b8f7e29c1.jpg  113462

[98440 rows x 4 columns]


In [32]:
model = tf.keras.applications.EfficientNetB2(include_top=True, weights=None, input_tensor=None, input_shape=(256, 256, 3), # "weights" is set to None, because the network will use the pretrained network's weights otherwise
    pooling=None, classes=200, classifier_activation='softmax')

In [33]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [34]:
model.summary()

Model: "efficientnetb2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 256, 256, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 256, 256, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization_1[0][0]            
_____________________________________________________________________________________

In [52]:
BATCH_SIZE = 16

In [53]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2, rescale=1.0/255.0)


train_gen = idg.flow_from_dataframe(
    train_keep,
    directory= train_dir +"/",
    x_col="filename",
    y_col="label",
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset="training",
    interpolation="nearest",
    validate_filenames=False)
    
val_gen = idg.flow_from_dataframe(
    train_keep,
    directory= train_dir +"/",
    x_col="filename",
    y_col="label",
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset="validation",
    interpolation="nearest",
    validate_filenames=False)

Found 78752 non-validated image filenames belonging to 200 classes.
Found 19688 non-validated image filenames belonging to 200 classes.


In [54]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_keep.landmark_id),
                                                 train_keep.landmark_id)
class_weights = dict(enumerate(class_weights))
class_weights

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[    27   1924   5170   6138   6208   9070   9673  10419  13471  13866
  14915  15445  18392  19605  20064  20102  20120  20409  21635  21703
  21843  25093  25457  26967  27190  27364  28139  28641  29794  31361
  31531  31837  31898  32338  33577  33636  33992  36748  38482  38494
  39209  39865  40088  41037  41648  41808  42016  42123  43845  44939
  45428  46705  47133  47378  47880  48753  51272  51856  53170  55219
  55350  56827  57015  57505  57943  60708  60730  64792  65068  65658
  65818  67406  67929  69835  70088  70644  70716  73211  73300  73304
  75005  76303  80147  80177  80272  83144  84939  85633  86869  87157
  90021  90396  90966  91274  96663  98993 100230 100631 101399 102850
 103899 105496 107164 107323 107801 108327 109169 110153 112308 113209
 113462 113636 113838 114289 115821 116124 116255 120144 120734 120885
 122418 124455 124675 125425 125786 126100 126637

{0: 0.9765873015873016,
 1: 0.5213983050847457,
 2: 1.5775641025641025,
 3: 1.582636655948553,
 4: 1.3521978021978023,
 5: 0.5468888888888889,
 6: 1.598051948051948,
 7: 0.6342783505154639,
 8: 1.4022792022792023,
 9: 1.4103151862464183,
 10: 0.6527851458885942,
 11: 1.15,
 12: 1.3375,
 13: 1.0004065040650407,
 14: 1.2817708333333333,
 15: 1.3825842696629214,
 16: 0.5542792792792792,
 17: 0.2799772468714448,
 18: 1.355922865013774,
 19: 1.4062857142857144,
 20: 1.5775641025641025,
 21: 0.7423831070889895,
 22: 1.4870090634441087,
 23: 1.5238390092879257,
 24: 0.9465384615384616,
 25: 1.6298013245033112,
 26: 1.0127572016460906,
 27: 1.177511961722488,
 28: 1.0630669546436284,
 29: 1.0962138084632516,
 30: 0.9746534653465346,
 31: 1.5098159509202453,
 32: 1.5098159509202453,
 33: 1.619078947368421,
 34: 1.5144615384615385,
 35: 1.5928802588996764,
 36: 1.4184438040345821,
 37: 0.5866507747318236,
 38: 0.6991477272727272,
 39: 1.2243781094527364,
 40: 1.4184438040345821,
 41: 1.088938053

In [55]:
# training parameters

EPOCHS = 20
train_steps = int(len(train_keep)*(1-0.2))//BATCH_SIZE
val_steps = int(len(train_keep)*0.2)//BATCH_SIZE

train_steps = train_steps * 0.2
val_steps = val_steps * 0.2

print(train_steps)
print(val_steps)

model_checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_categorical_accuracy', mode='max', verbose=1)

984.4000000000001
246.0


In [24]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [56]:
history = model.fit(train_gen, validation_data=val_gen, steps_per_epoch=train_steps, epochs = EPOCHS, validation_steps=val_steps, class_weight=class_weights, callbacks=[model_checkpoint])

model.save("model.h5")

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[16,32,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node efficientnetb2/block1a_bn/FusedBatchNormV3 (defined at <ipython-input-39-b8ec10f4a9b1>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_45281]

Function call stack:
train_function


In [ ]:
print(history.history.keys())
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#sub = pd.read_csv("/kaggle/input/landmark-recognition-2020/sample_submission.csv")
test_df["filename"] = test_df.id.str[0]+"/"+test_df.id.str[1]+"/"+test_df.id.str[2]+"/"+test_df.id+".jpg"

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/.255).flow_from_dataframe(
    test_df,
    directory= test_dir + "/",
    x_col="filename",
    y_col=None,
    weight_col=None,
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    batch_size=1,
    shuffle=True,
    subset=None,
    interpolation="nearest",
    validate_filenames=False)

y_pred = model.predict(test_gen, verbose=1)

In [ ]:
predict = model.predict(val_gen, val_steps,verbose=1)

good_preds = []
bad_preds = []

val_filenames = val_gen.filenames
label_map = (val_gen.class_indices)
#label_categories = to_categorical(np.asarray(labels)) 
cla = np.argmax(predict, axis=1)
label_map = list(map(int, label_map.keys()))
val_label = val_gen.labels

for idx, res in enumerate(predict):
    #print("image_id: ", val_filenames[idx], ", class predict: ", label_map[cla[idx]], "class: ", label_map[val_label[idx]])
    
    if label_map[cla[idx]] != label_map[val_label[idx]]:
        bad_preds.append([val_filenames[idx], label_map[cla[idx]], label_map[val_label[idx]], res[cla[idx]]])
    else:
        good_preds.append([val_filenames[idx], label_map[cla[idx]], label_map[val_label[idx]], res[cla[idx]]])
print("wrong predictions: ", len(bad_preds), " right predictions: ", len(good_preds), " acc: ", np.round(100*(len(predict)-len(bad_preds))/len(predict),2))

In [ ]:
### plot some of the best predictions
fig=plt.figure(figsize=(16, 8))

good_preds = np.array(good_preds)
good_preds = np.array(sorted(good_preds, key = lambda x: x[3], reverse=True))
#print(good_preds.shape)

columns = 4
rows = 1
for i in range(1, columns*rows +1):
    n = good_preds[i,0]
    #print(n)
    img = cv2.imread(os.path.join(train_dir,n))
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    lbl = good_preds[i,2]
    fig.add_subplot(rows, columns, i)
    plt.imshow(RGB_img)
    lbl2 = good_preds[i,1]
    plt.title("Label = " + str(lbl) + " \nClassified:" + str(lbl2) + " \nConfidence:" + str(good_preds[i,3]))
plt.show()

In [ ]:
### plot the worst predictions

fig=plt.figure(figsize=(16, 8))

bad_preds = np.array(bad_preds)
bad_preds = np.array(sorted(bad_preds, key = lambda x: x[3], reverse=True))
#print(bad_preds.shape)

columns = 4
rows = 1
for i in range(1, columns*rows +1):
    n = bad_preds[i,0]
    #print(n)
    img = cv2.imread(os.path.join(train_dir,n))
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    lbl = bad_preds[i,2]
    fig.add_subplot(rows, columns, i)
    plt.imshow(RGB_img)
    lbl2 = bad_preds[i,1]
    plt.title("Label = " + str(lbl) + " \nClassified:" + str(lbl2) + " \nConfidence:" + str(good_preds[i,3]))
plt.show()

In [ ]:
train_val = train_keep.landmark_id.value_counts()
train_keep_df = pd.DataFrame({'landmark_id':train_val.index, 'frequency':train_val.values})
train_keep_df.reset_index(inplace=True)

print("Top 5 training classes with most data:")
for i in range(5):
    print("label:", train_keep_df.landmark_id[i], "has", train_keep_df.frequency[i], "instances in training set" )

train_keep_df.set_index("landmark_id", inplace = True)
print("\nTop 5 classes with the worst prediction")
    
for i in range(5):
    label = bad_preds[i, 2]
    #print(label)
    label_counts = train_keep_df.loc[int(label)]
    #print(label_counts)
    print("label:", label, "has", label_counts["frequency"], "instances in training set" )
    
    print("\nTop 5 classes with the best prediction")
for i in range(5):
    label = good_preds[i, 2]
    #print(label)
    label_counts = train_keep_df.loc[int(label)]
    #print(label_counts)
    print("label:", label, "has", label_counts["frequency"], "instances in training set" )

In [ ]:
y_pred

In [ ]:
y_prob = np.max(y_pred, axis=-1)
y_pred = np.argmax(y_pred, axis=-1)
print(y_pred.shape, y_prob.shape)

In [ ]:
thresh = 0.5

for i in range(len(test_df)):
    prob = y_prob[i]
    pred = y_pred[i]
    if prob >= 0.3:
        st = str(pred)+" "+str(prob)
    else:
        st = ""
    test_df.loc[i, "landmarks"] = st
test_df = test_df.drop(columns="filename")
test_df.to_csv("submission.csv", index=False)
test_df

In [ ]:
prob

In [ ]:
model = keras.models.load_model("model.h5")

In [ ]:
history = model.fit(train_gen, validation_data=val_gen, steps_per_epoch=train_steps, epochs = 10, validation_steps=val_steps, class_weight=class_weights)



In [ ]:
print(history.history.keys())
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#sub = pd.read_csv("/kaggle/input/landmark-recognition-2020/sample_submission.csv")
test_df["filename"] = test_df.id.str[0]+"/"+test_df.id.str[1]+"/"+test_df.id.str[2]+"/"+test_df.id+".jpg"

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/.255).flow_from_dataframe(
    test_df,
    directory= test_dir + "/",
    x_col="filename",
    y_col=None,
    weight_col=None,
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    batch_size=1,
    shuffle=True,
    subset=None,
    interpolation="nearest",
    validate_filenames=False)

y_pred = model.predict(test_gen, verbose=1)

In [ ]:
predict = model.predict(val_gen, val_steps,verbose=1)

good_preds = []
bad_preds = []

val_filenames = val_gen.filenames
label_map = (val_gen.class_indices)
#label_categories = to_categorical(np.asarray(labels)) 
cla = np.argmax(predict, axis=1)
label_map = list(map(int, label_map.keys()))
val_label = val_gen.labels

for idx, res in enumerate(predict):
    #print("image_id: ", val_filenames[idx], ", class predict: ", label_map[cla[idx]], "class: ", label_map[val_label[idx]])
    
    if label_map[cla[idx]] != label_map[val_label[idx]]:
        bad_preds.append([val_filenames[idx], label_map[cla[idx]], label_map[val_label[idx]], res[cla[idx]]])
    else:
        good_preds.append([val_filenames[idx], label_map[cla[idx]], label_map[val_label[idx]], res[cla[idx]]])
print("wrong predictions: ", len(bad_preds), " right predictions: ", len(good_preds), " acc: ", np.round(100*(len(predict)-len(bad_preds))/len(predict),2))

In [ ]:
### plot some of the best predictions
fig=plt.figure(figsize=(16, 8))

good_preds = np.array(good_preds)
good_preds = np.array(sorted(good_preds, key = lambda x: x[3], reverse=True))
#print(good_preds.shape)

columns = 4
rows = 1
for i in range(1, columns*rows +1):
    n = good_preds[i,0]
    #print(n)
    img = cv2.imread(os.path.join(train_dir,n))
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    lbl = good_preds[i,2]
    fig.add_subplot(rows, columns, i)
    plt.imshow(RGB_img)
    lbl2 = good_preds[i,1]
    plt.title("Label = " + str(lbl) + " \nClassified:" + str(lbl2) + " \nConfidence:" + str(good_preds[i,3]))
plt.show()

In [ ]:
### plot the worst predictions

fig=plt.figure(figsize=(16, 8))

bad_preds = np.array(bad_preds)
bad_preds = np.array(sorted(bad_preds, key = lambda x: x[3], reverse=True))
#print(bad_preds.shape)

columns = 4
rows = 1
for i in range(1, columns*rows +1):
    n = bad_preds[i,0]
    #print(n)
    img = cv2.imread(os.path.join(train_dir,n))
    RGB_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    lbl = bad_preds[i,2]
    fig.add_subplot(rows, columns, i)
    plt.imshow(RGB_img)
    lbl2 = bad_preds[i,1]
    plt.title("Label = " + str(lbl) + " \nClassified:" + str(lbl2) + " \nConfidence:" + str(good_preds[i,3]))
plt.show()

In [ ]:
train_val = train_keep.landmark_id.value_counts()
train_keep_df = pd.DataFrame({'landmark_id':train_val.index, 'frequency':train_val.values})
train_keep_df.reset_index(inplace=True)

print("Top 5 training classes with most data:")
for i in range(5):
    print("label:", train_keep_df.landmark_id[i], "has", train_keep_df.frequency[i], "instances in training set" )

train_keep_df.set_index("landmark_id", inplace = True)
print("\nTop 5 classes with the worst prediction")
    
for i in range(5):
    label = bad_preds[i, 2]
    #print(label)
    label_counts = train_keep_df.loc[int(label)]
    #print(label_counts)
    print("label:", label, "has", label_counts["frequency"], "instances in training set" )
    
    print("\nTop 5 classes with the best prediction")
for i in range(5):
    label = good_preds[i, 2]
    #print(label)
    label_counts = train_keep_df.loc[int(label)]
    #print(label_counts)
    print("label:", label, "has", label_counts["frequency"], "instances in training set" )